# Preper the dataset



## SDNET2018: A benchmark dataset for semantic segmentation of crack images

https://digitalcommons.usu.edu/all_datasets/48/

Methodology
230 images of cracked and non-cracked concrete surfaces (54 bridge decks, 72 walls, 104 pavements) are captured using a 16 MP Nikon digital camera. The bridge decks were located at the Utah State University system, material, and structural health (SMASH) laboratory. The inspected walls belong to Russell/Wanlass Performance Hall building located on USU campus The pavement images were acquired from the roads and sidewalks on USU campus. Each image is segmented into 256 ◊ 256 px subimages. Each subimage is labeled as C if there was crack in the subimage or U if there was not a crack.


In [6]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class ConcretecrackDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.masks = []
        self.categories = []

        for category in ['D', 'P', 'W']:
            category_dir = os.path.join(root_dir, category)
            if os.path.isdir(category_dir):
                for class_name in os.listdir(category_dir):
                    class_dir = os.path.join(category_dir, class_name)
                    if os.path.isdir(class_dir):
                        for img_name in os.listdir(class_dir):
                            img_path = os.path.join(class_dir, img_name)
                            self.images.append(img_path)
                            # 创建一个简单的掩码，如果是裂缝图像，则整个图像都被标记为裂缝
                            mask = np.ones((256, 256)) if class_name.startswith('C') else np.zeros((256, 256))
                            self.masks.append(mask)
                            self.categories.append(category)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        mask = self.masks[idx]
        category = self.categories[idx]

        if self.transform:
            image = self.transform(image)
            mask = torch.from_numpy(mask).long()

        return image, mask, category

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 创建数据集实例
dataset = ConcretecrackDataset(root_dir='dataset/SDNET2018', transform=transform)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)



In [7]:
import torchvision.models as models

# 加载预训练的 DeepLabV3 模型
model = models.segmentation.deeplabv3_resnet101(pretrained=True)

# 修改最后一层以输出两个类（背景和裂缝）
model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))

# 将模型移至 GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


c:\Users\Administrator\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [8]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Train the network

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, masks, _ in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training finished!")



In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

def predict_and_visualize_crack(model, image_path):
    model.eval()
    
    # 加载和预处理图像
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)['out']
        output = F.softmax(output, dim=1)
        prediction = output.argmax(1).squeeze().cpu().numpy()

    # 创建一个掩码，其中裂缝像素为1，其他为0
    crack_mask = (prediction == 1).astype(np.uint8)

    # 使用形态学操作细化裂缝
    kernel = np.ones((3,3), np.uint8)
    crack_mask = cv2.morphologyEx(crack_mask, cv2.MORPH_OPEN, kernel)

    # 找到裂缝的轮廓
    contours, _ = cv2.findContours(crack_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 绘制原始图像
    plt.figure(figsize=(12, 4))
    plt.subplot(131)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    # 绘制预测的裂缝掩码
    plt.subplot(132)
    plt.imshow(crack_mask, cmap='gray')
    plt.title('Predicted Crack Mask')
    plt.axis('off')

    # 在原始图像上绘制裂缝轮廓
    plt.subplot(133)
    image_with_cracks = np.array(image)
    cv2.drawContours(image_with_cracks, contours, -1, (0, 255, 0), 2)
    plt.imshow(image_with_cracks)
    plt.title('Cracks Highlighted')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# 使用示例
predict_and_visualize_crack(model, 'path/to/test/image.jpg')


# Evaluate the network

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:  # 假设我们有一个测试数据加载器
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)['out']
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


# Predict the test set

In [ ]:
def predict_crack(image_path):
    model.eval()
    
    # 加载和预处理图像
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)['out']
        _, predicted = torch.max(output, 1)

    # 将预测结果转换为图像
    prediction = predicted.cpu().squeeze().numpy()
    
    return prediction

# 使用示例
result = predict_crack('path/to/test/image.jpg')
plt.imshow(result)
plt.show()
